In [253]:
import pandas as pd
import numpy as np

In [254]:
# количество итераций обучения
NUM_OF_ITERATIONS = 400

# читаем размеченные данные из файла
beer_dataset = pd.read_excel('beer_data_set.xlsx')

# печатаем список колонок таблички чтобы удобнее было копировать
print('\n'.join(beer_dataset.columns))

# выводим несколько первых строк таблицы
beer_dataset.head()

barcode
SKU_NAME
Наименование
Алкоголь
Объем
Производитель
Бренд
Саб-бренд
Тип упаковки
Мультипак
тип


,barcode,SKU_NAME,Наименование,Алкоголь,Объем,Производитель,Бренд,Саб-бренд,Тип упаковки,Мультипак,тип
0,4600721002206,Пиво БагБир 0.5л ст/бут,BAGBIER - светлое - 4.2% - 0.5л стекло,0.042,0.5,AB InBev,BAGBIER,NaN,Стекло,NaN,Светлое
1,4600721003197,Пиво БАГ-БИР св.ст/б 0.5л,BAGBIER - светлое - 4.2% - 0.5л стекло,0.042,0.5,AB InBev,BAGBIER,NaN,Стекло,NaN,Светлое
2,4600721003203,"Пиво BAGBIER светлое 4,9% 1.5л",BAGBIER - светлое - 4.9% - 1.5л пэт,0.049,1.5,AB InBev,BAGBIER,NaN,ПЭТ,NaN,Светлое
3,4600721005191,Пиво БАГ-БИР св.ПЭТ 2.5л,BAGBIER - светлое - 4.2% - 2.5л пэт,0.042,2.5,AB InBev,BAGBIER,NaN,ПЭТ,NaN,Светлое
4,4600721009366,"Пиво БАГ БИР ГОЛЬДЕН светлое ПЭТ 4% 1,5л",BAGBIER GOLDEN - светлое - 4.7% - 1.5л пэт,0.047,1.5,AB InBev,BAGBIER,Golden,ПЭТ,NaN,Светлое


Импортируем преобразователь текста в вектор.

[Примерное объяснение как работает](http://zabaykin.ru/?p=463)

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

In [255]:
from sklearn.feature_extraction.text import CountVectorizer

Импортируем преобразователь меток в числа. 
Он присваивает каждому новой встреченной метке уникальный номер.

Допустим у нас есть такой набор меток:
```python
labels = ['заяц', 'волк', 'утка', 'белка', 'заяц', 'утка']
```

LabelEncoder строит примерно такое соответсвие:

| Метка | Номер |
|:-----:|:-----:|
|  волк |   0   |
| белка |   1   |
| заяц  |   2   |
| утка  |   3   |

Так что если к нам поступает такой набор меток,
```python
['утка', 'утка', 'заяц', 'утка', 'белка', 'волк', 'заяц', 'волк', 'волк', 'белка']
```
то мы можем преобразовать их в список чисел
```python
[3, 3, 2, 3, 1, 0, 2, 0, 0, 1]
```

Это нужно потому что многие алгоритмы не умеют работать со строками, к тому же числа занимаю меньше места

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

In [256]:
from sklearn.preprocessing import LabelEncoder

Импортируем функцию для разбиения датасета на обучающий и тестирующий набор данных случайным образом. 

Зачем это нужно: когда модель обучается, то она может начать запоминать сочетания "вопрос"-"ответ". Вместо того, чтобы пытаться разобраться в во входных данных. Поэтому модель нужно проверять на данных, которых она до этого не видела. 

[Немного подробнее](http://robotosha.ru/algorithm/training-set-and-test-data.html)

[Документация](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [257]:
from sklearn.model_selection import train_test_split

Импортируем классификатор данных из библиотеки CatBoost от Яндекса

In [258]:
from catboost import CatBoostClassifier

Выдираем сырые строки из колонки **SKU_NAME**. В таблице строки хранятся в виде списков байт, метод `.flatten()` преобразует их в простые строки

In [259]:
data = [item for item in beer_dataset[['SKU_NAME']].values.flatten()]

Выдираем метки из колонки **Тип упаковки**. В таблице строки хранятся в виде списков байт, метод `.flatten()` преобразует их в простые строки. Так же на всякий случай переводим метки в нижний регистр с помощью метода `.lower()`

In [260]:
labels = [item.lower() for item in beer_dataset['Тип упаковки'].values.flatten()]

Разбиваем датасет случайным образом на обучающие и тестовые данные с соотношением 2 к 1. `random_state` – это начальное состояние генератора случайных чисел для разбиения, ставим число 42 чтобы каждый раз разбиение было одним и тем же.

In [261]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.33, random_state=42)

Печатаем статистику полученных датасетов: суммарный размер (Total), размер обучающей выборки (train) и тестовой (test)

In [262]:
print(f"Total: {len(data)} samples\n"
     f"\ttrain: {len(train_data)} data, {len(train_labels)} labels\n"
     f"\ttest: {len(test_data)} data, {len(test_labels)} labels")

Total: 5824 samples
	train: 3902 data, 3902 labels
	test: 1922 data, 1922 labels


Воспомогательная функция. CountVectorizer возвращает сжатые вектора, а нам нужны обычные. Эта функция берёт список сжатых веткоров и преобразует их в массивы чисел.

In [263]:
def dense_vectors(vectors):
    return [np.asarray(item.todense())[0] for item in vectors]

Функция обучения модели. Наша модель состоит из трёх частей:
    - `CountVectorizer` для преобразования входных данных в векторное представление
    - `LabelEncoder` для преобразования меток в числа
    - `CatBoostClassifier` – собственно классификатор данных

По-хорошему надо сохранять все три части в один или несколько файлов, пока для тестирования сохраняем только последнее. 

Входные данные в виде списка строк переводятся в нижний регистр, разбиваются на токены с помощью регулярного выражения `(?u)\b\w\w+\b|[0-9\.,]+[\%\w]|\w+`. Посмотреть как работает это выражение можно посмотреть [здесь](https://regex101.com/r/Puyk9J/1). Оно разбивает строки на список подстрок примерно так


|Строка| Токены |
|:------------------------------|:-----------------------------------:|
| Пиво БагБир 0.5л  ст/бут | ['Пиво', 'БагБир', '0.5л', 'ст', 'бут'] |
| Пиво БАГ-БИР св.ст/б 0.5л | ['Пиво', 'БАГ', 'БИР', 'св', '.с', 'т', 'б', '0.5л'] |
| Пиво BAGBIER светлое 4,9%   1.5л | ['Пиво', 'BAGBIER', 'светлое', '4,9%', '1.5л'] |
| Пиво БАГ-БИР св.ПЭТ 2.5л | ['Пиво', 'БАГ', 'БИР', 'св', '.П', 'ЭТ', '2.5л'] |
| Пиво БАГ БИР ГОЛЬДЕН светлое ПЭТ 4% 1,5л | ['Пиво', 'БАГ', 'БИР', 'ГОЛЬДЕН', 'светлое', 'ПЭТ', '4%', '1,5л'] |
| Пиво БАГ-БИР ГОЛЬДЕН св.4% ПЭТ 2.5л | ['Пиво', 'БАГ', 'БИР', 'ГОЛЬДЕН', 'св', '.4%', 'ПЭТ', '2.5л'] |
| Пиво БАГ БИР БОК темное 4% 1.5л | ['Пиво', 'БАГ', 'БИР', 'БОК', 'темное', '4%', '1.5л'] |
| Нап.пивн.BAGBIER 4,6% ст/б 0.5л | ['Нап', '.п', 'ивн', '.B', 'AGBIER', '4,6%', 'ст', 'б', '0.5л'] |
| Нап.пивн.БАГ БИР 4,2% ст/б 0.5л | ['Нап', '.п', 'ивн', '.Б', 'АГ', 'БИР', '4,2%', 'ст', 'б', '0.5л'] |
| Пиво BRAHMA 4.6% св.ст/б  0.33л | ['Пиво', 'BRAHMA', '4.6%', 'св', '.с', 'т', 'б', '0.33л'] |

Потом мы переводим данные в сжатые вектора, а из них получаем простые вектора с помощью фунции `dense_vectors`. Затем создаём LabelEncoder() и заставляем его запомнить наши метки.

Теперь можно заняться самым главным: обучить классификатор. 

Мы создаём CatBoostClassifier с настраиваемым параметром iterations (число итераций) и обучаем его через вызов метода `fit` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_fit.html)). Этот метод принимает обучающие данные в виде списка векторов (`vectorized_data`) и список закодированных меток.

После того, как обучились, возвращаем кортеж вида `(CountVectorizer, LabelEncoder, CatBoostClassifier)`

In [264]:
def build_model(data, labels, iterations=200):
        vectorizer = CountVectorizer(lowercase=True, 
                                     analyzer="word",
                                    token_pattern=r"(?u)\b\w\w+\b|[0-9\.,]+[\%\w]|\w+")
        
        compressed_data = vectorizer.fit_transform(data)
        vectorized_data = dense_vectors(compressed_data)
        
        le = LabelEncoder()
        encoded_labels = le.fit_transform(labels)
            
        classifier = CatBoostClassifier(iterations=iterations, task_type = "GPU")
        classifier.fit(vectorized_data, encoded_labels, silent=False)
        
        return (vectorizer, le, classifier)


Обучаем нашу модель

In [265]:
model = build_model(train_data, train_labels, iterations=NUM_OF_ITERATIONS)

0:	learn: -1.5139363	total: 20.1ms	remaining: 8.02s
1:	learn: -1.4297616	total: 36.6ms	remaining: 7.27s
2:	learn: -1.3553701	total: 56.7ms	remaining: 7.51s
3:	learn: -1.2889445	total: 76.5ms	remaining: 7.58s
4:	learn: -1.2291322	total: 94.3ms	remaining: 7.45s
5:	learn: -1.1753842	total: 121ms	remaining: 7.96s
6:	learn: -1.1257986	total: 141ms	remaining: 7.93s
7:	learn: -1.0803537	total: 169ms	remaining: 8.3s
8:	learn: -1.0385213	total: 189ms	remaining: 8.22s
9:	learn: -0.9998674	total: 207ms	remaining: 8.06s
10:	learn: -0.9640308	total: 228ms	remaining: 8.07s
11:	learn: -0.9321800	total: 252ms	remaining: 8.13s
12:	learn: -0.9009999	total: 270ms	remaining: 8.03s
13:	learn: -0.8728040	total: 288ms	remaining: 7.94s
14:	learn: -0.8471110	total: 306ms	remaining: 7.84s
15:	learn: -0.8211580	total: 322ms	remaining: 7.74s
16:	learn: -0.7976255	total: 340ms	remaining: 7.67s
17:	learn: -0.7745975	total: 358ms	remaining: 7.6s
18:	learn: -0.7519506	total: 376ms	remaining: 7.55s
19:	learn: -0.73127

159:	learn: -0.1873961	total: 4.34s	remaining: 6.51s
160:	learn: -0.1872535	total: 4.36s	remaining: 6.47s
161:	learn: -0.1869017	total: 4.38s	remaining: 6.43s
162:	learn: -0.1861017	total: 4.39s	remaining: 6.39s
163:	learn: -0.1859648	total: 4.41s	remaining: 6.35s
164:	learn: -0.1854871	total: 4.43s	remaining: 6.31s
165:	learn: -0.1852234	total: 4.45s	remaining: 6.27s
166:	learn: -0.1847975	total: 4.46s	remaining: 6.23s
167:	learn: -0.1846339	total: 4.49s	remaining: 6.2s
168:	learn: -0.1845323	total: 4.51s	remaining: 6.16s
169:	learn: -0.1842440	total: 4.53s	remaining: 6.13s
170:	learn: -0.1836624	total: 4.55s	remaining: 6.09s
171:	learn: -0.1832332	total: 4.57s	remaining: 6.05s
172:	learn: -0.1828042	total: 4.58s	remaining: 6.02s
173:	learn: -0.1825653	total: 4.61s	remaining: 5.99s
174:	learn: -0.1824170	total: 4.63s	remaining: 5.96s
175:	learn: -0.1822706	total: 4.65s	remaining: 5.92s
176:	learn: -0.1821394	total: 4.67s	remaining: 5.89s
177:	learn: -0.1816871	total: 4.7s	remaining: 5

322:	learn: -0.1599876	total: 8.9s	remaining: 2.12s
323:	learn: -0.1599348	total: 8.92s	remaining: 2.09s
324:	learn: -0.1595587	total: 8.94s	remaining: 2.06s
325:	learn: -0.1595276	total: 8.96s	remaining: 2.03s
326:	learn: -0.1593526	total: 8.98s	remaining: 2s
327:	learn: -0.1592437	total: 8.99s	remaining: 1.97s
328:	learn: -0.1590725	total: 9.01s	remaining: 1.95s
329:	learn: -0.1590341	total: 9.03s	remaining: 1.92s
330:	learn: -0.1590013	total: 9.06s	remaining: 1.89s
331:	learn: -0.1588138	total: 9.07s	remaining: 1.86s
332:	learn: -0.1587659	total: 9.09s	remaining: 1.83s
333:	learn: -0.1587294	total: 9.11s	remaining: 1.8s
334:	learn: -0.1585648	total: 9.13s	remaining: 1.77s
335:	learn: -0.1585218	total: 9.14s	remaining: 1.74s
336:	learn: -0.1583910	total: 9.16s	remaining: 1.71s
337:	learn: -0.1583547	total: 9.18s	remaining: 1.68s
338:	learn: -0.1582157	total: 9.2s	remaining: 1.65s
339:	learn: -0.1581838	total: 9.21s	remaining: 1.63s
340:	learn: -0.1581427	total: 9.23s	remaining: 1.6s


Генератор отчёта по нашей модели. Скармливаем ей модель, тестовые данные и метки.

Метод `.predict` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_predict.html)) классификатора `CountVectorizer` принимает список векторов (входные данные) и возвращает для них самые вероятные ответы в виде чисел float, которые нужно преобразовать к целым числам.

Метод `.predict_proba` ([документация](https://catboost.ai/docs/concepts/python-reference_catboostclassifier_predict_proba.html)) классификатора `CountVectorizer` принимает список векторов (входные данные) и возвращает для них вероятности полученных выше ответов. Полученные вероятности умножем на 100 и преобразумем в строки вида `95%, 80%, 91%`. 

После этого берём полученные из метода `.predict` ответы и раскодируем их с помощью метода `.inverse_transform` кодировщика `LabelEncoder`, который преобразует список чисел в список строк.

Из полученных выше ответов, входных данных и правильных ответов из тестового датасета делаем табличку (`table`) и преобразуем её в удобный для нас вид `pandas.DataFrame`.

In [266]:
def validate_model(model, valid_data, valid_labels):
    vectorizer, le, classifier = model
    columns = ["Запись", "Предсказание", "Уверенность" ,"Правильный результат"]

    compressed_data = vectorizer.transform(valid_data)
    vectorized_data = dense_vectors(compressed_data)
    prediction = classifier.predict(vectorized_data).flatten().astype('int64')
    
    proba = np.rint(100*classifier.predict_proba(vectorized_data).max(axis=1))
    proba_column = (f"{int(item)}%" for item in proba)
    
    results = le.inverse_transform(prediction).flatten()
    table = zip(valid_data, results, proba_column, valid_labels)
    return pd.DataFrame(table, columns=columns)

Запускаем валидацию нашей модели и выводим сравнительную табличку результатов

In [267]:
validation = validate_model(model, test_data, test_labels)
validation.head(20)

,Запись,Предсказание,Уверенность,Правильный результат
0,"Пиво СТЕПАН РАЗИН ПЕТР.св.4,7% ПЭТ 1.4л",пэт,97%,пэт
1,ZLATY BAZANT - светлое 4.1% - 0.5л ж.б.,ж.б.,98%,ж.б.
2,"Пиво MILLER MIDNIGHT тем.4,8% ст/б 0.5л",стекло,97%,стекло
3,КЛИНСКОЕ - светлое 4.5% - 0.5л стекло,стекло,97%,стекло
4,"ПивоANDECH.ВАЙСБ.ХЕФЕТР.св.5,5%ст/б 0.5л",стекло,97%,стекло
5,ЖИГУЛЕВСКОЕ - светлое 4.2% - 1.5л пэт,пэт,97%,пэт
6,"Пиво ЯРПИВО КРЕПК.св. 7,2% 0.5л",стекло,91%,ж.б.
7,БАЛТИКА - светлое 5.4% - 0.5л стекло,стекло,97%,стекло
8,SIBIRSKAYA KORONA(AB INBEV) KLASSICHESKOE 0.5L...,ж.б.,92%,ж.б.
9,OKHOTA(HEINEKEN BEL) KREPKOE EKSPORTNOE 1.4L P...,пэт,94%,пэт


Хорошо бы посчитать процент правильных ответов. Фильтруем все строки, в которых значение в столбце **Предсказание** совпадает со значением в столбце **Правильный результат** и считаем их количество.

Делим число правильных ответов на размер тестового набора данных и выводим.

In [268]:
valid = len(validation[validation['Предсказание'] == validation['Правильный результат']])
total = len(validation)
print(f"Valid: {valid} from {total} ({100*valid/total}%)")


Valid: 1829 from 1922 (95.16129032258064%)


Не забываем сохранить модель в файл!

In [269]:
vectorizer, le, classifier = model

classifier.save_model(f"beer_container_catboost_{NUM_OF_ITERATIONS}_iterations.cbm")